In [56]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import json
import pandas as pd

In [53]:
data_path = "../data/domain_bound_data/"

## stanford-question-answering-dataset - Generate Q & A dataset

In [34]:

# Set the path to the file you'd like to load
file_path = "train-v1.1.json"

# Load the latest version
general_qna_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "stanfordu/stanford-question-answering-dataset",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)


/tmp/ipykernel_64037/1706377438.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  general_qna_df = kagglehub.load_dataset(


In [35]:
general_qna_df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [36]:
for title in general_qna_df["data"].apply(lambda data: data["title"]).sort_values():
    print(title)

2008_Sichuan_earthquake
2008_Summer_Olympics_torch_relay
51st_state
ASCII
A_cappella
Adolescence
Adult_contemporary_music
Affirmative_action_in_the_United_States
Age_of_Enlightenment
Aircraft_carrier
Airport
Alaska
Alexander_Graham_Bell
Alfred_North_Whitehead
Alloy
Alps
Alsace
American_Idol
Animal
Ann_Arbor,_Michigan
Annelid
Antarctica
Antenna_(radio)
Anthropology
Anti-aircraft_warfare
Antibiotics
Apollo
Appalachian_Mountains
Architecture
Arena_Football_League
Armenia
Armenians
Arnold_Schwarzenegger
Arsenal_F.C.
Ashkenazi_Jews
Asphalt
Aspirated_consonant
Association_football
Asthma
Athanasius_of_Alexandria
Atlantic_City,_New_Jersey
Avicenna
BBC_Television
Bacteria
Baptists
Beer
BeiDou_Navigation_Satellite_System
Bermuda
Bern
Beyoncé
Bill_%26_Melinda_Gates_Foundation
Biodiversity
Bird
Bird_migration
Black_people
Boston
Botany
Brain
Bras%C3%ADlia
Brigham_Young_University
British_Empire
British_Isles
Buckingham_Palace
Buddhism
CBC_Television
Canadian_Armed_Forces
Canadian_football
Canon_l

In [31]:
medical_healthcare_topics = {
    "Antibiotics",
    "Genome",
    "Gene",
    "Brain",
    "Immunology",
    "Digestion",
    "Myocardial_infarction",
    "Tuberculosis",
    "Asthma",
    "Diarrhea",
    "Pain",
    "Bacteria",
    "Infection",
    "Circadian_rhythm",
    "Pharmaceutical_industry",
    "Nutrition",
    "Human_Development_Index",
}

In [47]:
general_qna_df_filtered = general_qna_df[general_qna_df["data"].apply(lambda data: data["title"] not in medical_healthcare_topics)]

In [49]:
general_qna_df_filtered.iloc[0, 0]["paragraphs"][0]["qas"][0]["question"]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [51]:
questions = []

def find_questions(data):
    for item in data["paragraphs"]:
        for qas in item["qas"]:
            questions.append(qas["question"])

general_qna_df_filtered["data"].apply(find_questions)
questions

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?',
 'When did the Scholastic Magazine of Notre dame begin publishing?',
 "How often is Notre Dame's the Juggler published?",
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'In what year did the student paper Common Sense begin publication at Notre Dame?',
 'Where is the headquarters of the Congregation of the Holy Cross?',
 'What is the primary seminary of the Congregation of the Holy Cross?',
 'What is the oldest structure at Notre Dame?',
 'What individuals live at Fatima House at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'How many BS level degrees are offered in the College of Engineering at Notre

In [54]:
len(questions)

84847

In [58]:
non_med_df = pd.DataFrame({"question": questions})

In [59]:
non_med_df.head()

,question
0,To whom did the Virgin Mary allegedly appear i...
1,What is in front of the Notre Dame Main Building?
2,The Basilica of the Sacred heart at Notre Dame...
3,What is the Grotto at Notre Dame?
4,What sits on top of the Main Building at Notre...


In [60]:
non_med_df = non_med_df.sample(frac=1, random_state=32).reset_index(drop=True)
non_med_df.head()

,question
0,What was the name of the explorer who visited ...
1,What mountains are the city near?
2,What are some of the empires and dynasties tha...
3,Which team member has his own fan following?
4,What does http stand for?


## Coding questions

In [64]:
import csv

def fix_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Read and write header
        headers = next(reader)
        writer.writerow(headers)
        
        # Process each row
        for row in reader:
            if len(row) >= 2:
                # Write ID as-is, Question with quotes
                writer.writerow([row[0], f'"{row[1]}"'])
            else:
                writer.writerow(row)

# Example usage:
fix_csv(data_path + 'coding_questions.txt', data_path + 'coding_questions.csv')

In [77]:
def f(s):
    if s.startswith('"'):
        s = s[1:]
    if s.endswith('"'):
        s = s[:-1]
    return s
coding_df["question"] = coding_df["question"].apply(f)
coding_df.to_csv(data_path + "coding_questions.csv")

In [78]:
coding_df = pd.read_csv(data_path + "coding_questions.csv")
coding_df.head()

,Unnamed: 0,ID,question
0,0,1,Write a Python function that takes a list of d...
1,1,2,Generate a JavaScript function that asynchrono...
2,2,3,Create a SQL query that joins three tables (Users
3,3,4,Write a Python script that uses multithreading...
4,4,5,Generate a Java method that reads a large CSV ...


In [70]:
coding_df = coding_df.rename(columns={"Question": "question"})

In [94]:
non_med_qna_df = pd.concat([non_med_df.iloc[:1500, :], coding_df["question"]])
non_med_qna_df = non_med_qna_df.sample(frac=1, random_state=32).reset_index(drop=True)
non_med_qna_df.head(30)

,question
0,What did women commonly have less of?
1,Write a Python function to check if a year is ...
2,What Queen song did the Glee choir perform in ...
3,What did Viacom plan to do after the ruling in...
4,Did Bortolazzi like playing the new wire strun...
5,In what movie was Mr. White originally suppose...
6,The torch route covered six what?
7,What were some of the names given to language ...
8,What was Schwarzenegger's mother's maiden name?
9,Nintendo required what from its software devel...


In [95]:
non_med_qna_df.shape

(1800, 1)

In [85]:
non_med_responses_df = pd.read_csv(data_path + "non_med_responses.csv")

In [93]:
non_med_responses_df.sample().iloc[0, 0]

'I can’t provide information unrelated to health.'

In [99]:
non_med_qna_df["answer"] = non_med_qna_df["question"].apply(lambda _: non_med_responses_df.sample().iloc[0, 0])

In [101]:
# add the class
non_med_qna_df["class"] = non_med_qna_df["question"].apply(lambda _: "non_med")

In [102]:
non_med_qna_df.head()

,question,answer,class
0,What did women commonly have less of?,My responses are limited to health and medical...,non_med
1,Write a Python function to check if a year is ...,That topic falls outside the scope of my train...,non_med
2,What Queen song did the Glee choir perform in ...,This model is trained to provide accurate heal...,non_med
3,What did Viacom plan to do after the ruling in...,This model is designed for healthcare professi...,non_med
4,Did Bortolazzi like playing the new wire strun...,"Please ask about symptoms, treatments, or heal...",non_med


In [106]:
non_med_qna_df.to_csv(data_path + "non_med_qna.csv")

# Medical domain questions

In [97]:
med_qna_df = pd.read_csv(data_path + "qna_long_seq_filtered.csv")
med_qna_df = med_qna_df.loc[112300:, ["question", "answer"]]
med_qna_df.head()

,question,answer
112300,What are the treatments for adermatoglyphia ?,These resources address the diagnosis or manag...
112301,Im trying to get pregnant and my Gynecologist ...,"Hi, Tamoxifen is for increasing growth of your..."
112302,"Hi, just wondering does Kalms ( a natural reme...","Hi, There is no documented evidence suggestive..."
112303,Who is at risk for Colorectal Cancer? ?,Different factors increase or decrease the ris...
112304,"I have a discomfort feeling on my right side, ...","Hi, Thanks for asking. Based on your query, my..."


In [98]:
med_qna_df.shape

(9654, 2)

In [103]:
med_qna_df = med_qna_df.iloc[:2000, :].reset_index(drop=True)
med_qna_df.head()

,question,answer
0,What are the treatments for adermatoglyphia ?,These resources address the diagnosis or manag...
1,Im trying to get pregnant and my Gynecologist ...,"Hi, Tamoxifen is for increasing growth of your..."
2,"Hi, just wondering does Kalms ( a natural reme...","Hi, There is no documented evidence suggestive..."
3,Who is at risk for Colorectal Cancer? ?,Different factors increase or decrease the ris...
4,"I have a discomfort feeling on my right side, ...","Hi, Thanks for asking. Based on your query, my..."


In [104]:
med_qna_df.shape

(2000, 2)

In [105]:
med_qna_df["class"] = med_qna_df["question"].apply(lambda _: "med")
med_qna_df.head()

,question,answer,class
0,What are the treatments for adermatoglyphia ?,These resources address the diagnosis or manag...,med
1,Im trying to get pregnant and my Gynecologist ...,"Hi, Tamoxifen is for increasing growth of your...",med
2,"Hi, just wondering does Kalms ( a natural reme...","Hi, There is no documented evidence suggestive...",med
3,Who is at risk for Colorectal Cancer? ?,Different factors increase or decrease the ris...,med
4,"I have a discomfort feeling on my right side, ...","Hi, Thanks for asking. Based on your query, my...",med


In [107]:
med_qna_df.to_csv(data_path + "med_qna.csv")

## Combined data set

In [108]:
domain_bound_qna_df = pd.concat([non_med_qna_df, med_qna_df])
domain_bound_qna_df.head()

,question,answer,class
0,What did women commonly have less of?,My responses are limited to health and medical...,non_med
1,Write a Python function to check if a year is ...,That topic falls outside the scope of my train...,non_med
2,What Queen song did the Glee choir perform in ...,This model is trained to provide accurate heal...,non_med
3,What did Viacom plan to do after the ruling in...,This model is designed for healthcare professi...,non_med
4,Did Bortolazzi like playing the new wire strun...,"Please ask about symptoms, treatments, or heal...",non_med


In [109]:
domain_bound_qna_df = domain_bound_qna_df.sample(frac=1, random_state=32).reset_index(drop=True)

In [112]:
domain_bound_qna_df.shape

(3800, 3)

In [113]:
train_df = domain_bound_qna_df.iloc[:3100, :]
test_df = domain_bound_qna_df.iloc[3100:3600, :]
val_df = domain_bound_qna_df.iloc[3600:, :]

In [114]:
train_df.shape, test_df.shape, val_df.shape

((3100, 3), (500, 3), (200, 3))

In [115]:
train_df.to_csv(data_path + "train.csv", index=False)
test_df.to_csv(data_path + "test.csv", index=False)
val_df.to_csv(data_path + "val.csv", index=False)